In [1]:
library('SelectSim')
library('tidyverse')

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lubridate_1.9.3   forcats_1.0.0     stringr_1.5.1     dplyr_1.1.4      
 [5] purrr_1.0.2       readr_2.1.5       tidyr_1.3.0       tibble_3.2.1     
 [9] ggplot2_3.4.4     tidyverse_2.0.0   SelectSim_0.0.1.3

loaded via a nam

#  Functions

In [3]:
# Filter the pan_can_run data object to required needs.
filter_run_data <- function(run_data,gene_order,sample_order){
    final_run_data<-run_data
    final_run_data$M$M$missense<-run_data$M$M$missense[gene_order,sample_order]
    final_run_data$M$M$truncating<-run_data$M$M$truncating[gene_order,sample_order]
    final_run_data$alteration.class<- run_data$alteration.class[gene_order]
    final_run_data$sample.class<-run_data$sample.class[sample_order]
    final_run_data$M$tmb$missense<-run_data$M$tmb$missense[sample_order,]
    final_run_data$M$tmb$truncating<-run_data$M$tmb$truncating[sample_order,]
    return(final_run_data)
}

# Metadata

In [4]:
all_merged_metdata <- readRDS(file='/mnt/ndata/arvind/co_mutation_project/data/processed/all_merged_data_genie_v15.rds')
genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples<-readRDS(file='/mnt/ndata/arvind/co_mutation_project/data/processed/genie/dfci_one_patient_one_sample_final.rds')

In [5]:
dfci_run_gene_list <- readRDS(file='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/data/dfci_oncokb_gene_run_list.rds')

In [6]:
head(all_merged_metdata,1)

,sample,class,tumor_run,oncotree,cancer_type,race,cohort,tumor_run_mapped,Tissue,final_tumor_run_mapped
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TCGA-OR-A5J5-01,ACC,ACC,ACC,Adrenocortical Carcinoma,White,TCGA,No mapping,Adrenal Gland,ACC


In [7]:
str(dfci_run_gene_list)

List of 4
 $ all: chr [1:177] "MTOR" "NRAS" "NTRK1" "SDHC" ...
 $ p_2: chr [1:194] "MTOR" "NRAS" "CD58" "NTRK1" ...
 $ p_3: chr [1:257] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_1: chr [1:183] "MTOR" "NRAS" "CD58" "NTRK1" ...


# Pan-Cancer Runs

##  DFCI Primary

In [8]:
dfci_run_p_data <- readRDS('/mnt/ndata/arvind/co_mutation_project/data/processed/gams/pan_can_dfci_primary_run_data_v15.rds')

In [9]:
str(dfci_run_p_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:14657] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:14657] "GENIE-DFCI-000018-4015" "GENIE-DFCI-003388-5143" "GENIE-DFCI-001376-5688" "GENIE-DFCI-001329-1827" ...
  .. ..$ truncating: num [1:396, 1:14657] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:14657] "GENIE-DFCI-000018-4015" "GENIE-DFCI-003388-5143" "GENIE-DFCI-001376-5688" "GENIE-DFCI-001329-1827" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	14657 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:14657] "GENIE-DFCI-000018-4015" "GENIE-DFCI-003388-5143" "GENIE-DFCI-001376-5688" "GENIE-DFCI-001329-1827" ...
  .. .. ..$ mutation: num [1:14657] 6 9 3 5 1 1 7 5 0 2 ...
  .. ..$ truncating:'data.frame':	14657 obs. of  2 variabl

In [10]:
table(dfci_run_p_data$sample.class)
length(table(dfci_run_p_data$sample.class))


             AML             ANSC             BLCA             CHOL 
             122               74              469              137 
          CLLSLL           CRC_HM           CRC_LM             DIFG 
             136              294             1990              100 
        DLBCLNOS             ESCA              GBM            GINET 
              95              664             1423               93 
            GIST Glioma_low_grade             HEMA           HGGNOS 
             215              309              173              160 
           HGSOC             HNSC              IDC             IHCH 
             335              343              957               98 
             ILC           LGGNOS              LMS             LUAD 
             214              111              101             1658 
            LUSC              MCC             MDLC              MEL 
             322              100               98              357 
             MNG            NSCLC

[1] 43

In [11]:
str(dfci_run_p_data$sample.class)

 Named chr [1:14657] "BLCA" "LUSC" "THPA" "UCEC_LM" "SCLC" "LUAD" "HNSC" ...
 - attr(*, "names")= chr [1:14657] "GENIE-DFCI-000018-4015" "GENIE-DFCI-003388-5143" "GENIE-DFCI-001376-5688" "GENIE-DFCI-001329-1827" ...


In [12]:
genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% head(1)

,PATIENT_ID,SEX,PRIMARY_RACE,ETHNICITY,CENTER,INT_CONTACT,INT_DOD,YEAR_CONTACT,DEAD,YEAR_DEATH,SAMPLE_ID,AGE_AT_SEQ_REPORT,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_TYPE_DETAILED,final_class,final_run_group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GENIE-DFCI-000145,Male,White,Non-Spanish/non-Hispanic,DFCI,26418,Not Applicable,2022,False,Not Applicable,GENIE-DFCI-000145-3971,64,LUAD,Metastasis,DFCI-ONCOPANEL-2,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,Metastasis site unspecified,LUAD,Non-Small Cell Lung Cancer


In [13]:
str(dfci_run_gene_list)

List of 4
 $ all: chr [1:177] "MTOR" "NRAS" "NTRK1" "SDHC" ...
 $ p_2: chr [1:194] "MTOR" "NRAS" "CD58" "NTRK1" ...
 $ p_3: chr [1:257] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_1: chr [1:183] "MTOR" "NRAS" "CD58" "NTRK1" ...


In [14]:
assay_id<-unique(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples$SEQ_ASSAY_ID)

In [15]:
assay_id

[1] "DFCI-ONCOPANEL-2"   "DFCI-ONCOPANEL-1"   "DFCI-ONCOPANEL-3.1"
[4] "DFCI-ONCOPANEL-3"

In [ ]:
for(name in names(dfci_run_gene_list)){
    cat('--Running:',name,"--\n")
    if(name=='all'){
        samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% filter(SAMPLE_TYPE=='Primary'))$SAMPLE_ID
        temp<- filter_run_data(run_data = dfci_run_p_data,
                               gene_order = dfci_run_gene_list$all,
                               sample_order =samples ) 
        label<-paste('../data/','dfci_panel_',name,'_primary_run_data.rds',sep="")
        saveRDS(temp,file = label,compress = TRUE)
        dfci_result <- selectX(M = temp$M,
                            sample.class = temp$sample.class,
                            alteration.class = temp$alteration.class,
                            n.cores = 15,
                            min.freq = round(0.001*length(temp$sample.class)),
                            n.permut = 1000,
                            lambda = 0.3,
                            tao = 1,
                            save.object = FALSE,
                            verbose = FALSE,
                            estimate_pairwise = FALSE,
                            maxFDR = 0.25,
                            seed = 42)
        label<-paste('../results/','dfci_panel_',name,'_primary_result.rds',sep="")
        saveRDS(dfci_result,file = label,compress = TRUE)
        rm(dfci_result)
    }
    else if(name=='p_2'){
        samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                  filter(SAMPLE_TYPE=='Primary' &SEQ_ASSAY_ID=='DFCI-ONCOPANEL-2') %>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
        temp<- filter_run_data(run_data = dfci_run_p_data,
                               gene_order = dfci_run_gene_list$p_2,
                               sample_order =samples) 
        label<-paste('../data/','dfci_panel_',name,'_primary_run_data.rds',sep="")
        saveRDS(temp,file = label,compress = TRUE)
        dfci_result <- selectX(M = temp$M,
                            sample.class = temp$sample.class,
                            alteration.class = temp$alteration.class,
                            n.cores = 15,
                            min.freq = round(0.001*length(temp$sample.class)),
                            n.permut = 1000,
                            lambda = 0.3,
                            tao = 1,
                            save.object = FALSE,
                            verbose = FALSE,
                            estimate_pairwise = FALSE,
                            maxFDR = 0.25,
                            seed = 42)

        label<-paste('../results/','dfci_panel_',name,'_primary_result.rds',sep="")
        saveRDS(dfci_result,file = label,compress = TRUE)
        rm(dfci_result)
    }
    else if(name=='p_1'){
        samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                  filter(SAMPLE_TYPE=='Primary' &SEQ_ASSAY_ID=='DFCI-ONCOPANEL-1') %>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
        temp<- filter_run_data(run_data = dfci_run_p_data,
                               gene_order = dfci_run_gene_list$p_1,
                               sample_order =samples) 
        label<-paste('../data/','dfci_panel_',name,'_primary_run_data.rds',sep="")
        saveRDS(temp,file = label,compress = TRUE)
        dfci_result <- selectX(M = temp$M,
                            sample.class = temp$sample.class,
                            alteration.class = temp$alteration.class,
                            n.cores = 15,
                            min.freq = round(0.001*length(temp$sample.class)),
                            n.permut = 1000,
                            lambda = 0.3,
                            tao = 1,
                            save.object = FALSE,
                            verbose = FALSE,
                            estimate_pairwise = FALSE,
                            maxFDR = 0.25,
                            seed = 42)

        label<-paste('../results/','dfci_panel_',name,'_primary_result.rds',sep="")
        saveRDS(dfci_result,file = label,compress = TRUE)
        rm(dfci_result)
    }
    else{
        samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
        filter(SAMPLE_TYPE=='Primary' & SEQ_ASSAY_ID %in% c('DFCI-ONCOPANEL-3.1','DFCI-ONCOPANEL-3')) %>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
        temp<- filter_run_data(run_data = dfci_run_p_data,
                               gene_order = dfci_run_gene_list$p_3,
                               sample_order =samples) 
        name<-'p3'
        label<-paste('../data/','dfci_panel_',name,'_primary_run_data.rds',sep="")
        saveRDS(temp,file = label,compress = TRUE)
        dfci_result <- selectX(M = temp$M,
                            sample.class = temp$sample.class,
                            alteration.class = temp$alteration.class,
                            n.cores = 15,
                            min.freq = round(0.001*length(temp$sample.class)),
                            n.permut = 1000,
                            lambda = 0.3,
                            tao = 1,
                            save.object = FALSE,
                            verbose = FALSE,
                            estimate_pairwise = FALSE,
                            maxFDR = 0.25,
                            seed = 42)

        label<-paste('../results/','dfci_panel_',name,'_primary_result.rds',sep="")
        saveRDS(dfci_result,file = label,compress = TRUE)
        rm(dfci_result)
    }
 cat('\n---Done---\n')
}

## DFCI Metastatsis 

In [16]:
dfci_run_m_data <- readRDS('/mnt/ndata/arvind/co_mutation_project/data/processed/gams/pan_can_dfci_meta_run_data_v15.rds')

In [17]:
str(dfci_run_m_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:7607] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:7607] "GENIE-DFCI-000145-3971" "GENIE-DFCI-007383-3930" "GENIE-DFCI-000544-5758" "GENIE-DFCI-008041-5875" ...
  .. ..$ truncating: num [1:396, 1:7607] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:7607] "GENIE-DFCI-000145-3971" "GENIE-DFCI-007383-3930" "GENIE-DFCI-000544-5758" "GENIE-DFCI-008041-5875" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	7607 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:7607] "GENIE-DFCI-000145-3971" "GENIE-DFCI-007383-3930" "GENIE-DFCI-000544-5758" "GENIE-DFCI-008041-5875" ...
  .. .. ..$ mutation: num [1:7607] 2 5 11 1 2 4 4 5 4 5 ...
  .. ..$ truncating:'data.frame':	7607 obs. of  2 variables:
  .

In [18]:
table(dfci_run_m_data$sample.class)
length(table(dfci_run_m_data$sample.class))


             AML             ANSC             BLCA             CHOL 
               9               26              144               55 
          CLLSLL           CRC_HM           CRC_LM             DIFG 
               4               51              846               13 
        DLBCLNOS             ESCA              GBM            GINET 
              27              159              212               75 
            GIST Glioma_low_grade             HEMA           HGGNOS 
              90               74                1               25 
           HGSOC             HNSC              IDC             IHCH 
             550               90              985               14 
             ILC           LGGNOS              LMS             LUAD 
             353                9               93             1243 
            LUSC              MCC             MDLC              MEL 
             124               57               16              584 
             MNG            NSCLC

[1] 43

In [19]:
str(dfci_run_m_data$sample.class)

 Named chr [1:7607] "LUAD" "LUAD" "LUAD" "STAD" "IDC" "UCEC_LM" "IDC" "IDC" ...
 - attr(*, "names")= chr [1:7607] "GENIE-DFCI-000145-3971" "GENIE-DFCI-007383-3930" "GENIE-DFCI-000544-5758" "GENIE-DFCI-008041-5875" ...


In [20]:
genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% head(1)

,PATIENT_ID,SEX,PRIMARY_RACE,ETHNICITY,CENTER,INT_CONTACT,INT_DOD,YEAR_CONTACT,DEAD,YEAR_DEATH,SAMPLE_ID,AGE_AT_SEQ_REPORT,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_TYPE_DETAILED,final_class,final_run_group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GENIE-DFCI-000145,Male,White,Non-Spanish/non-Hispanic,DFCI,26418,Not Applicable,2022,False,Not Applicable,GENIE-DFCI-000145-3971,64,LUAD,Metastasis,DFCI-ONCOPANEL-2,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,Metastasis site unspecified,LUAD,Non-Small Cell Lung Cancer


In [21]:
str(dfci_run_gene_list)

List of 4
 $ all: chr [1:177] "MTOR" "NRAS" "NTRK1" "SDHC" ...
 $ p_2: chr [1:194] "MTOR" "NRAS" "CD58" "NTRK1" ...
 $ p_3: chr [1:257] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_1: chr [1:183] "MTOR" "NRAS" "CD58" "NTRK1" ...


In [22]:
assay_id<-unique(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples$SEQ_ASSAY_ID)

In [23]:
assay_id

[1] "DFCI-ONCOPANEL-2"   "DFCI-ONCOPANEL-1"   "DFCI-ONCOPANEL-3.1"
[4] "DFCI-ONCOPANEL-3"

In [ ]:
for(name in names(dfci_run_gene_list)){
    cat('--Running:',name,"--\n")
    if(name=='all'){
        samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% filter(SAMPLE_TYPE!='Primary') %>% 
                  group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
        temp<- filter_run_data(run_data = dfci_run_m_data,
                               gene_order = dfci_run_gene_list$all,
                               sample_order =samples ) 
        label<-paste('../data/','dfci_panel_',name,'_meta_run_data.rds',sep="")
        saveRDS(temp,file = label,compress = TRUE)
        dfci_result <- selectX(M = temp$M,
                            sample.class = temp$sample.class,
                            alteration.class = temp$alteration.class,
                            n.cores = 15,
                            min.freq = round(0.001*length(temp$sample.class)),
                            n.permut = 1000,
                            lambda = 0.3,
                            tao = 1,
                            save.object = FALSE,
                            verbose = FALSE,
                            estimate_pairwise = FALSE,
                            maxFDR = 0.25,
                            seed = 42)
        label<-paste('../results/','dfci_panel_',name,'_meta_result.rds',sep="")
        saveRDS(dfci_result,file = label,compress = TRUE)
        rm(dfci_result)
    }
    else if(name=='p_2'){
        samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                  filter(SAMPLE_TYPE!='Primary' &SEQ_ASSAY_ID=='DFCI-ONCOPANEL-2') %>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
        temp<- filter_run_data(run_data = dfci_run_m_data,
                               gene_order = dfci_run_gene_list$p_2,
                               sample_order =samples) 
        label<-paste('../data/','dfci_panel_',name,'_meta_run_data.rds',sep="")
        saveRDS(temp,file = label,compress = TRUE)
        dfci_result <- selectX(M = temp$M,
                            sample.class = temp$sample.class,
                            alteration.class = temp$alteration.class,
                            n.cores = 15,
                            min.freq = round(0.001*length(temp$sample.class)),
                            n.permut = 1000,
                            lambda = 0.3,
                            tao = 1,
                            save.object = FALSE,
                            verbose = FALSE,
                            estimate_pairwise = FALSE,
                            maxFDR = 0.25,
                            seed = 42)

        label<-paste('../results/','dfci_panel_',name,'_meta_result.rds',sep="")
        saveRDS(dfci_result,file = label,compress = TRUE)
        rm(dfci_result)
    }
    else if(name=='p_1'){
        samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                  filter(SAMPLE_TYPE!='Primary' &SEQ_ASSAY_ID=='DFCI-ONCOPANEL-1') %>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
        temp<- filter_run_data(run_data = dfci_run_m_data,
                               gene_order = dfci_run_gene_list$p_1,
                               sample_order =samples) 
        label<-paste('../data/','dfci_panel_',name,'_meta_run_data.rds',sep="")
        saveRDS(temp,file = label,compress = TRUE)
        dfci_result <- selectX(M = temp$M,
                            sample.class = temp$sample.class,
                            alteration.class = temp$alteration.class,
                            n.cores = 15,
                            min.freq = round(0.001*length(temp$sample.class)),
                            n.permut = 1000,
                            lambda = 0.3,
                            tao = 1,
                            save.object = FALSE,
                            verbose = FALSE,
                            estimate_pairwise = FALSE,
                            maxFDR = 0.25,
                            seed = 42)

        label<-paste('../results/','dfci_panel_',name,'_meta_result.rds',sep="")
        saveRDS(dfci_result,file = label,compress = TRUE)
        rm(dfci_result)
    }
    else{
        samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
        filter(SAMPLE_TYPE!='Primary' & SEQ_ASSAY_ID %in% c('DFCI-ONCOPANEL-3.1','DFCI-ONCOPANEL-3')) %>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
        temp<- filter_run_data(run_data = dfci_run_m_data,
                               gene_order = dfci_run_gene_list$p_3,
                               sample_order =samples) 
        name<-'p3'
        label<-paste('../data/','dfci_panel_',name,'_meta_run_data.rds',sep="")
        saveRDS(temp,file = label,compress = TRUE)
        dfci_result <- selectX(M = temp$M,
                            sample.class = temp$sample.class,
                            alteration.class = temp$alteration.class,
                            n.cores = 15,
                            min.freq = round(0.001*length(temp$sample.class)),
                            n.permut = 1000,
                            lambda = 0.3,
                            tao = 1,
                            save.object = FALSE,
                            verbose = FALSE,
                            estimate_pairwise = FALSE,
                            maxFDR = 0.25,
                            seed = 42)

        label<-paste('../results/','dfci_panel_',name,'_meta_result.rds',sep="")
        saveRDS(dfci_result,file = label,compress = TRUE)
        rm(dfci_result)
    }
 cat('\n---Done---\n')
}

# Tumor Group Runs

## DFCI Primary

In [24]:
genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% head(1)

,PATIENT_ID,SEX,PRIMARY_RACE,ETHNICITY,CENTER,INT_CONTACT,INT_DOD,YEAR_CONTACT,DEAD,YEAR_DEATH,SAMPLE_ID,AGE_AT_SEQ_REPORT,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_TYPE_DETAILED,final_class,final_run_group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GENIE-DFCI-000145,Male,White,Non-Spanish/non-Hispanic,DFCI,26418,Not Applicable,2022,False,Not Applicable,GENIE-DFCI-000145-3971,64,LUAD,Metastasis,DFCI-ONCOPANEL-2,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,Metastasis site unspecified,LUAD,Non-Small Cell Lung Cancer


In [25]:
str(dfci_run_gene_list)

List of 4
 $ all: chr [1:177] "MTOR" "NRAS" "NTRK1" "SDHC" ...
 $ p_2: chr [1:194] "MTOR" "NRAS" "CD58" "NTRK1" ...
 $ p_3: chr [1:257] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_1: chr [1:183] "MTOR" "NRAS" "CD58" "NTRK1" ...


In [26]:
assay_id<-unique(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples$SEQ_ASSAY_ID)

In [27]:
assay_id

[1] "DFCI-ONCOPANEL-2"   "DFCI-ONCOPANEL-1"   "DFCI-ONCOPANEL-3.1"
[4] "DFCI-ONCOPANEL-3"

In [28]:
genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% head(2)

,PATIENT_ID,SEX,PRIMARY_RACE,ETHNICITY,CENTER,INT_CONTACT,INT_DOD,YEAR_CONTACT,DEAD,YEAR_DEATH,SAMPLE_ID,AGE_AT_SEQ_REPORT,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_TYPE_DETAILED,final_class,final_run_group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GENIE-DFCI-000145,Male,White,Non-Spanish/non-Hispanic,DFCI,26418,Not Applicable,2022,False,Not Applicable,GENIE-DFCI-000145-3971,64,LUAD,Metastasis,DFCI-ONCOPANEL-2,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,Metastasis site unspecified,LUAD,Non-Small Cell Lung Cancer
2,GENIE-DFCI-007383,Male,White,Non-Spanish/non-Hispanic,DFCI,23668,23673,2015,True,2015,GENIE-DFCI-007383-3930,64,LUAD,Metastasis,DFCI-ONCOPANEL-2,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,Metastasis site unspecified,LUAD,Non-Small Cell Lung Cancer


In [29]:
tumor_groups<-unique(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples$final_run_group)

In [30]:
cleaned_tumor_groups <- tolower(gsub("[[:space:]/-]", "_", tumor_groups))

In [31]:
mapped_tumor_groups <- cleaned_tumor_groups
names(mapped_tumor_groups)<-tumor_groups

In [32]:
str(mapped_tumor_groups)

 Named chr [1:26] "non_small_cell_lung_cancer" "bladder_cancer" ...
 - attr(*, "names")= chr [1:26] "Non-Small Cell Lung Cancer" "Bladder Cancer" "Thyroid Cancer" "Esophagogastric Cancer" ...


In [ ]:
for(name in names(dfci_run_gene_list)){
    cat('--Running:',name,"--\n")
    if(name=='all'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE=='Primary' & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    temp<- filter_run_data(run_data = dfci_run_p_data,
                                           gene_order = dfci_run_gene_list$all,
                                           sample_order =samples ) 
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste('../data/','dfci_panel_',finalname,'_primary_run_data.rds',sep="")
                    saveRDS(temp,file = label,compress = TRUE)
                    dfci_result <- selectX(M = temp$M,
                                        sample.class = temp$sample.class,
                                        alteration.class = temp$alteration.class,
                                        n.cores = 15,
                                        min.freq = round(0.001*length(temp$sample.class)),
                                        n.permut = 1000,
                                        lambda = 0.3,
                                        tao = 1,
                                        save.object = FALSE,
                                        verbose = FALSE,
                                        estimate_pairwise = FALSE,
                                        maxFDR = 0.25,
                                        seed = 42)
                    label<-paste('../results/','dfci_panel_',finalname,'_primary_result.rds',sep="")
                    saveRDS(dfci_result,file = label,compress = TRUE)
                    rm(dfci_result)                
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }    
    }
    else if(name=='p_2'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE=='Primary' &SEQ_ASSAY_ID=='DFCI-ONCOPANEL-2' & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    temp<- filter_run_data(run_data = dfci_run_p_data,
                                           gene_order = dfci_run_gene_list$p_2,
                                           sample_order =samples ) 
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste('../data/','dfci_panel_',finalname,'_primary_run_data.rds',sep="")
                    saveRDS(temp,file = label,compress = TRUE)
                    dfci_result <- selectX(M = temp$M,
                                        sample.class = temp$sample.class,
                                        alteration.class = temp$alteration.class,
                                        n.cores = 15,
                                        min.freq = round(0.001*length(temp$sample.class)),
                                        n.permut = 1000,
                                        lambda = 0.3,
                                        tao = 1,
                                        save.object = FALSE,
                                        verbose = FALSE,
                                        estimate_pairwise = FALSE,
                                        maxFDR = 0.25,
                                        seed = 42)
                    label<-paste('../results/','dfci_panel_',finalname,'_primary_result.rds',sep="")
                    saveRDS(dfci_result,file = label,compress = TRUE)
                    rm(dfci_result)                
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }
    }
    else if(name=='p_1'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE=='Primary' &SEQ_ASSAY_ID=='DFCI-ONCOPANEL-1' & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    temp<- filter_run_data(run_data = dfci_run_p_data,
                                           gene_order = dfci_run_gene_list$p_1,
                                           sample_order =samples ) 
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste('../data/','dfci_panel_',finalname,'_primary_run_data.rds',sep="")
                    saveRDS(temp,file = label,compress = TRUE)
                    dfci_result <- selectX(M = temp$M,
                                        sample.class = temp$sample.class,
                                        alteration.class = temp$alteration.class,
                                        n.cores = 15,
                                        min.freq = round(0.001*length(temp$sample.class)),
                                        n.permut = 1000,
                                        lambda = 0.3,
                                        tao = 1,
                                        save.object = FALSE,
                                        verbose = FALSE,
                                        estimate_pairwise = FALSE,
                                        maxFDR = 0.25,
                                        seed = 42)
                    label<-paste('../results/','dfci_panel_',finalname,'_primary_result.rds',sep="")
                    saveRDS(dfci_result,file = label,compress = TRUE)
                    rm(dfci_result)                
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }
        
    }
    else{
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE=='Primary' & SEQ_ASSAY_ID %in% c('DFCI-ONCOPANEL-3.1','DFCI-ONCOPANEL-3') & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    temp<- filter_run_data(run_data = dfci_run_p_data,
                                           gene_order = dfci_run_gene_list$p_3,
                                           sample_order =samples ) 
                    finalname<-paste(mapped_tumor_groups[tumor_group],'p_3',sep="_")
                    label<-paste('../data/','dfci_panel_',finalname,'_primary_run_data.rds',sep="")
                    saveRDS(temp,file = label,compress = TRUE)
                    dfci_result <- selectX(M = temp$M,
                                        sample.class = temp$sample.class,
                                        alteration.class = temp$alteration.class,
                                        n.cores = 15,
                                        min.freq = round(0.001*length(temp$sample.class)),
                                        n.permut = 1000,
                                        lambda = 0.3,
                                        tao = 1,
                                        save.object = FALSE,
                                        verbose = FALSE,
                                        estimate_pairwise = FALSE,
                                        maxFDR = 0.25,
                                        seed = 42)
                    label<-paste('../results/','dfci_panel_',finalname,'_primary_result.rds',sep="")
                    saveRDS(dfci_result,file = label,compress = TRUE)
                    rm(dfci_result)                
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }
    }
 cat('\n---Done---\n')
}

## DFCI Metastasis

In [33]:
genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% head(1)

,PATIENT_ID,SEX,PRIMARY_RACE,ETHNICITY,CENTER,INT_CONTACT,INT_DOD,YEAR_CONTACT,DEAD,YEAR_DEATH,SAMPLE_ID,AGE_AT_SEQ_REPORT,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_TYPE_DETAILED,final_class,final_run_group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GENIE-DFCI-000145,Male,White,Non-Spanish/non-Hispanic,DFCI,26418,Not Applicable,2022,False,Not Applicable,GENIE-DFCI-000145-3971,64,LUAD,Metastasis,DFCI-ONCOPANEL-2,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,Metastasis site unspecified,LUAD,Non-Small Cell Lung Cancer


In [34]:
str(dfci_run_gene_list)

List of 4
 $ all: chr [1:177] "MTOR" "NRAS" "NTRK1" "SDHC" ...
 $ p_2: chr [1:194] "MTOR" "NRAS" "CD58" "NTRK1" ...
 $ p_3: chr [1:257] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_1: chr [1:183] "MTOR" "NRAS" "CD58" "NTRK1" ...


In [35]:
assay_id<-unique(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples$SEQ_ASSAY_ID)

In [36]:
assay_id

[1] "DFCI-ONCOPANEL-2"   "DFCI-ONCOPANEL-1"   "DFCI-ONCOPANEL-3.1"
[4] "DFCI-ONCOPANEL-3"

In [37]:
genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% head(2)

,PATIENT_ID,SEX,PRIMARY_RACE,ETHNICITY,CENTER,INT_CONTACT,INT_DOD,YEAR_CONTACT,DEAD,YEAR_DEATH,SAMPLE_ID,AGE_AT_SEQ_REPORT,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_TYPE_DETAILED,final_class,final_run_group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GENIE-DFCI-000145,Male,White,Non-Spanish/non-Hispanic,DFCI,26418,Not Applicable,2022,False,Not Applicable,GENIE-DFCI-000145-3971,64,LUAD,Metastasis,DFCI-ONCOPANEL-2,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,Metastasis site unspecified,LUAD,Non-Small Cell Lung Cancer
2,GENIE-DFCI-007383,Male,White,Non-Spanish/non-Hispanic,DFCI,23668,23673,2015,True,2015,GENIE-DFCI-007383-3930,64,LUAD,Metastasis,DFCI-ONCOPANEL-2,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,Metastasis site unspecified,LUAD,Non-Small Cell Lung Cancer


In [38]:
tumor_groups<-unique(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples$final_run_group)

In [39]:
cleaned_tumor_groups <- tolower(gsub("[[:space:]/-]", "_", tumor_groups))

In [40]:
mapped_tumor_groups <- cleaned_tumor_groups
names(mapped_tumor_groups)<-tumor_groups

In [41]:
str(mapped_tumor_groups)

 Named chr [1:26] "non_small_cell_lung_cancer" "bladder_cancer" ...
 - attr(*, "names")= chr [1:26] "Non-Small Cell Lung Cancer" "Bladder Cancer" "Thyroid Cancer" "Esophagogastric Cancer" ...


In [ ]:
for(name in names(dfci_run_gene_list)){
    cat('--Running:',name,"--\n")
    if(name=='all'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE!='Primary' & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    temp<- filter_run_data(run_data = dfci_run_m_data,
                                           gene_order = dfci_run_gene_list$all,
                                           sample_order =samples ) 
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste('../data/','dfci_panel_',finalname,'_meta_run_data.rds',sep="")
                    saveRDS(temp,file = label,compress = TRUE)
                    dfci_result <- selectX(M = temp$M,
                                        sample.class = temp$sample.class,
                                        alteration.class = temp$alteration.class,
                                        n.cores = 15,
                                        min.freq = round(0.001*length(temp$sample.class)),
                                        n.permut = 1000,
                                        lambda = 0.3,
                                        tao = 1,
                                        save.object = FALSE,
                                        verbose = FALSE,
                                        estimate_pairwise = FALSE,
                                        maxFDR = 0.25,
                                        seed = 42)
                    label<-paste('../results/','dfci_panel_',finalname,'_meta_result.rds',sep="")
                    saveRDS(dfci_result,file = label,compress = TRUE)
                    rm(dfci_result)                
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }    
    }
    else if(name=='p_2'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE!='Primary' &SEQ_ASSAY_ID=='DFCI-ONCOPANEL-2' & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    temp<- filter_run_data(run_data = dfci_run_m_data,
                                           gene_order = dfci_run_gene_list$p_2,
                                           sample_order =samples ) 
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste('../data/','dfci_panel_',finalname,'_meta_run_data.rds',sep="")
                    saveRDS(temp,file = label,compress = TRUE)
                    dfci_result <- selectX(M = temp$M,
                                        sample.class = temp$sample.class,
                                        alteration.class = temp$alteration.class,
                                        n.cores = 15,
                                        min.freq = round(0.001*length(temp$sample.class)),
                                        n.permut = 1000,
                                        lambda = 0.3,
                                        tao = 1,
                                        save.object = FALSE,
                                        verbose = FALSE,
                                        estimate_pairwise = FALSE,
                                        maxFDR = 0.25,
                                        seed = 42)
                    label<-paste('../results/','dfci_panel_',finalname,'_meta_result.rds',sep="")
                    saveRDS(dfci_result,file = label,compress = TRUE)
                    rm(dfci_result)                
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }
    }
    else if(name=='p_1'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE!='Primary' &SEQ_ASSAY_ID=='DFCI-ONCOPANEL-1' & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    temp<- filter_run_data(run_data = dfci_run_m_data,
                                           gene_order = dfci_run_gene_list$p_1,
                                           sample_order =samples ) 
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste('../data/','dfci_panel_',finalname,'_meta_run_data.rds',sep="")
                    saveRDS(temp,file = label,compress = TRUE)
                    dfci_result <- selectX(M = temp$M,
                                        sample.class = temp$sample.class,
                                        alteration.class = temp$alteration.class,
                                        n.cores = 15,
                                        min.freq = round(0.001*length(temp$sample.class)),
                                        n.permut = 1000,
                                        lambda = 0.3,
                                        tao = 1,
                                        save.object = FALSE,
                                        verbose = FALSE,
                                        estimate_pairwise = FALSE,
                                        maxFDR = 0.25,
                                        seed = 42)
                    label<-paste('../results/','dfci_panel_',finalname,'_meta_result.rds',sep="")
                    saveRDS(dfci_result,file = label,compress = TRUE)
                    rm(dfci_result)                
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }
        
    }
    else{
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_dfci_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE!='Primary' & SEQ_ASSAY_ID %in% c('DFCI-ONCOPANEL-3.1','DFCI-ONCOPANEL-3') & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    temp<- filter_run_data(run_data = dfci_run_m_data,
                                           gene_order = dfci_run_gene_list$p_3,
                                           sample_order =samples ) 
                    finalname<-paste(mapped_tumor_groups[tumor_group],'p_3',sep="_")
                    label<-paste('../data/','dfci_panel_',finalname,'_meta_run_data.rds',sep="")
                    saveRDS(temp,file = label,compress = TRUE)
                    dfci_result <- selectX(M = temp$M,
                                        sample.class = temp$sample.class,
                                        alteration.class = temp$alteration.class,
                                        n.cores = 15,
                                        min.freq = round(0.001*length(temp$sample.class)),
                                        n.permut = 1000,
                                        lambda = 0.3,
                                        tao = 1,
                                        save.object = FALSE,
                                        verbose = FALSE,
                                        estimate_pairwise = FALSE,
                                        maxFDR = 0.25,
                                        seed = 42)
                    label<-paste('../results/','dfci_panel_',finalname,'_meta_result.rds',sep="")
                    saveRDS(dfci_result,file = label,compress = TRUE)
                    rm(dfci_result)                
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }
    }
 cat('\n---Done---\n')
}